In [ ]:
from cell_paint_seg import utils, image_io
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from skimage import exposure
import numpy as np
from skimage import measure

In [ ]:
# List of channel names ordered by channel number
channels = ["Brightfield", "ER", "AGP", "Mito", "DNA", "RNA"]

# Directories where images and predicted segmentations are located
path_dir_im = "C:\\Users\\zeiss\\projects\\athey_als\\test-images\\tifs" 
path_dir_seg_ilastik = "C:\\Users\\zeiss\\projects\\athey_als\\test-images\\segmentations"

path_dir_im = "D:\Aneesh\Assay Dev 20230329\BR00142687__2024-03-29T18_18_57-Measurement 1\Images"
path_dir_seg_ilastik = "D:\Aneesh\Assay Dev 20230329\BR00142687__2024-03-29T18_18_57-Measurement 1\segmentations"

reg_stat_limits = {"area": (-1, 4000)}


# Substrings that identify whether a file should be collected
tag_im = ".tiff"
tag_seg_ilastik = ".tif"
tag_seg_cell = "ch7"
tag_seg_soma = "ch8"
tag_seg_nuc = "ch9"

# View results

In [ ]:
id_to_path_im = utils.get_id_to_path(path_dir_im)
id_to_path_seg_cell = utils.get_id_to_path(path_dir_seg_ilastik, tag_seg_cell)
id_to_path_seg_soma = utils.get_id_to_path(path_dir_seg_ilastik, tag_seg_soma)
id_to_path_seg_nuc = utils.get_id_to_path(path_dir_seg_ilastik, tag_seg_nuc)

for n_id, id in enumerate(id_to_path_im.keys()):
    print(id)
    images = image_io.read_ims(id_to_path_im[id])


    seg_cell = image_io.read_seg(id_to_path_seg_cell[id])
    seg_nuc = image_io.read_seg(id_to_path_seg_nuc[id])
    seg_soma = image_io.read_seg(id_to_path_seg_soma[id])
    seg_cyto = np.logical_and(seg_soma > 0, seg_nuc == 0)

    seg_cell_masked = np.ma.masked_array(seg_cell, mask=seg_cell == 0)
    seg_soma_masked = np.ma.masked_array(seg_soma, mask=seg_soma == 0)
    seg_nuc_masked = np.ma.masked_array(seg_nuc, mask=seg_nuc == 0)
    seg_cyto_masked = np.ma.masked_array(seg_cyto, mask=seg_cyto == 0)


    image_dna = images[channels.index("DNA")]
    image_dna = exposure.equalize_adapthist(image_dna, clip_limit=0.03, kernel_size = [s // 64 for s in image_dna.shape])
    image_rna = images[channels.index("RNA")]
    image_rna = exposure.equalize_adapthist(image_rna, clip_limit=0.03)
    image_agp = images[channels.index("AGP")]
    image_agp = exposure.equalize_adapthist(image_agp, clip_limit=0.03)
    image_rgb = np.stack([image_agp, image_rna, image_dna], axis=2)


    f, axs = plt.subplots(nrows=1, ncols=2)
    axs[0].imshow(image_rgb, cmap="gray")
    axs[0].set_title(f"Image {id}", fontsize=24)

    axs[1].imshow(image_dna, cmap="gray")
    axs[1].imshow(seg_cell_masked % 20, cmap="tab20", alpha=0.5)
    axs[1].imshow(seg_cyto_masked, cmap="Accent", alpha=0.7)
    axs[1].imshow(seg_nuc_masked > 0, cmap="Set1", alpha=0.7)
    # axs[1].imshow(seg_cyto_masked, cmap="Accent", alpha=0.7)
    # axs[1].imshow(seg_nuc_masked > 0, cmap="Set1", alpha=0.7)
    axs[1].set_title(f"{seg_cell.max()} Cells Detected", fontsize=24)
 
    for ax in axs.flatten():
        ax.axis("off")
        
    f.set_size_inches(30, 10)
    f.tight_layout()
    plt.show()

    if n_id > 5:
        break